# Importing the necessary libraries

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re, nltk, spacy, string
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from nltk.corpus import wordnet
import en_core_web_sm
nlp = en_core_web_sm.load()
#EDA

from collections import Counter
from nltk.util import ngrams
from wordcloud import WordCloud


#nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# Load the dataset

In [ ]:
df = pd.read_csv("/content/processed_reviews (3).csv")

In [ ]:
df.shape# givies the shape of the dataset

(5655, 2)

In [ ]:
df.head()# displays he first 5 rows in the dataset

,category,cleaned_review_body
0,Positive,great experience atom start easy process open ...
1,Positive,straight forward clear instruction offering go...
2,Positive,done doddle happy atom interest rate best took...
3,Positive,happy service provided atom app clear easy pro...
4,Positive,app user friendly customer care friendly effic...


# Transformer model

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.7 MB/s eta 0:00:00


Installing a tranformers as we are using GPT2 model to train the data

## import the packages that will be used for  Tensorflow project.

In [ ]:
import math
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2Model

# feature extraction

In [ ]:
# Text preprocessing and feature extraction (TF-IDF)
X = df['cleaned_review_body']
y = df['category']
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(X)

Splitting the dataset into train and test

# Handling imbalanced data

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer


smote = SMOTE(random_state=42)

# SMOTE to balance the dataset
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

#  class distribution after applying SMOTE
print("Class distribution after SMOTE:", Counter(y_resampled))



X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, stratify=y_resampled)

Class distribution after SMOTE: Counter({'Positive': 5405, 'Neutral': 5405, 'Negative': 5405})


In [ ]:
X_train.shape


(12161, 4174)

In [ ]:
X_train_dense = X_train.toarray()#to cal the maxlength convert x_train into array

# Calculate the maximum sequence length
maxlen = np.max(np.array([len(str(x).split()) for x in X_train_dense])) + 2

The code calculates the average word length in X_train and then adds 2 to round up to the nearest integer using math.ceil

initializing  the tokenizer and add a PAD token and EOS (end of text) token to maintain the fixed lengths in the tokens.

In [ ]:
pad = "<|pad|>" #padding tokens
eos = "<|endoftext|>"#end of sequence tokens

# download and initialize the pre trained tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2",
    pad_token=pad,
    eos_token=eos,
    max_length=maxlen,
    is_split_into_words=True) # the input  are already tokenised into words hence set to True

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


the above code is used to configure and initialize a tokenizer for the GPT-2 language model from the Hugging Face Transformers library.

In [ ]:
X_train = [str(ex) + eos for ex in X_train]# Modify training data by appending the 'eos' token to each example
X_test = [str(ex) + eos for ex in X_test]# Modify testing data by appending the 'eos' token to each example

In [ ]:
# Tokenize and preprocess the training data using the GPT-2 tokenizer
X_train_ = [tokenizer(str(x), return_tensors='tf', max_length=maxlen, truncation=True, pad_to_max_length=True, add_special_tokens=True)['input_ids'] for x in X_train]
# Tokenize and preprocess the testing data using the GPT-2 tokenizer
X_test_ = [tokenizer(str(x), return_tensors='tf', max_length=maxlen, truncation=True, pad_to_max_length=True, add_special_tokens=True)['input_ids'] for x in X_test]


# Convert the tokenized training data into TensorFlow tensors and squeeze any unnecessary dimensions
X_train_in = tf.squeeze(tf.convert_to_tensor(X_train_), axis=1)
# Convert the tokenized testing data into TensorFlow tensors and squeeze any unnecessary dimensions

X_test_in = tf.squeeze(tf.convert_to_tensor(X_test_), axis=1)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
# Tokenize and preprocess the training data to obtain attention masks using the GPT-2 tokenizer
X_train_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=maxlen, truncation=True, pad_to_max_length=True, add_special_tokens=True)["attention_mask"] for x in X_train]
# Tokenize and preprocess the testing data to obtain attention masks using the GPT-2 tokenizer
X_test_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=maxlen, truncation=True, pad_to_max_length=True, add_special_tokens=True)["attention_mask"] for x in X_test]


# Convert the attention masks of the training data into TensorFlow tensors and squeeze any unnecessary dimensions
X_train_mask = tf.squeeze(tf.convert_to_tensor(X_train_mask_), axis=1)
# Convert the attention masks of the testing data into TensorFlow tensors and squeeze any unnecessary dimensions
X_test_mask = tf.squeeze(tf.convert_to_tensor(X_test_mask_), axis=1)

Here the training data is tokenised and preprocessed using GPT2 tokenizer to get the mask. This mask is used to identify which input sequence is valid and which is pad sequence.

The mask data are later converted into tensorflow and unnecessary dimensions are reduced.

## Fine tuning GPT2 model

In [ ]:
# Load a pre-trained GPT-2 model for fine-tuning, with caching disabled to maintain the model consistency.
model = TFGPT2Model.from_pretrained("gpt2", use_cache=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id)
model.training = True# to set the model in training mode

All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


 the model will update its weights and gradients during training, allowing it to learn from the data,hence we need to set the model.training=True

In [ ]:
model.resize_token_embeddings(len(tokenizer)) # Resize the token embeddings of the model to match the vocabulary size of the tokenizer

In [ ]:
# Set all layers in the model to non-trainable
for layer in model.layers:
    layer.trainable = False#avoid overfitting

In [ ]:
model.summary()# used to display the summary of architecture

Model: "tfgpt2_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  124440576 
 er)                                                             
                                                                 
Total params: 124440576 (474.70 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 124440576 (474.70 MB)
_________________________________________________________________


In [ ]:
input = tf.keras.layers.Input(shape=(None,), dtype='int32')# input layer for the model, which takes integer sequences as input
mask = tf.keras.layers.Input(shape=(None,), dtype='int32')# input layer for attention masks
x = model(input, attention_mask=mask)
x = tf.reduce_mean(x.last_hidden_state, axis=1)#  the mean of the last hiddesn states across the sequence dimension
x = tf.keras.layers.Dense(128, activation='relu')(x)# dense layer with ReLU activation function to increase learning of model
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(3, activation='softmax')(x)

In [ ]:
clf=tf.keras.Model(inputs=[input, mask], outputs=output)

In [ ]:
clf.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 tfgpt2_model (TFGPT2Model)  TFBaseModelOutputWithPastA   1244405   ['input_1[0][0]',             
                             ndCrossAttentions(last_hid   76         'input_2[0][0]']             
                             den_state=(None, None, 768                                           
                             ),                                                               

In [ ]:
base_learning_rate = 0.001 # the base learning rate
optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate)# adam optimizer
loss=tf.keras.losses.SparseCategoricalCrossentropy()

# Compile the model with the optimizer and loss function
clf.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
callbacks = tf.keras.callbacks.EarlyStopping(
        monitor="accuracy", verbose=1, patience=3, restore_best_weights=True)# EarlyStopping callback

The EarlyStopping callback is useful for preventing overfitting and for saving time during training when further training doesn't lead to improvements in accuracy.

In [ ]:
def map_sentiment(value):
  if value == 'Negative':
    return 0
  if value == 'Neutral':
    return 1
  if value == 'Positive':
    return 2


mapping the sentiment values. if negative then 0, if neutral then 1, if positive then 2.

In [ ]:
# Apply a mapping function to transform the labels in y_train and y_test
y_train_ = y_train.map(map_sentiment)
y_test_= y_test.map(map_sentiment)

In [ ]:
# Convert transformed labels into TensorFlow constants
y_train_in = tf.constant(y_train_, dtype=tf.int32)
y_test_in = tf.constant(y_test_, dtype=tf.int32)

In [ ]:
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


Eager execution is a mode in TensorFlow where operations are executed immediately as they are called, rather than being added to a computational graph for later execution.

In [ ]:
#fitting the model
history = clf.fit([X_train_in, X_train_mask], y_train_in, epochs=10, batch_size=64, validation_split=0.2, callbacks=callbacks)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


152/152 [==============================] - 92s 602ms/step - loss: 2.0111 - accuracy: 0.3572 - val_loss: 1.0711 - val_accuracy: 0.4254
Epoch 2/10
152/152 [==============================] - 88s 581ms/step - loss: 1.0682 - accuracy: 0.4118 - val_loss: 1.0361 - val_accuracy: 0.4755
Epoch 3/10
152/152 [==============================] - 91s 597ms/step - loss: 1.0550 - accuracy: 0.4232 - val_loss: 1.0207 - val_accuracy: 0.4533
Epoch 4/10
152/152 [==============================] - 87s 571ms/step - loss: 1.0384 - accuracy: 0.4407 - val_loss: 1.0423 - val_accuracy: 0.4628
Epoch 5/10
152/152 [==============================] - 88s 580ms/step - loss: 1.0281 - accuracy: 0.4575 - val_loss: 1.0155 - val_accuracy: 0.4390
Epoch 6/10
152/152 [==============================] - 90s 596ms/step - loss: 1.0191 - accuracy: 0.4694 - val_loss: 1.0030 - val_accuracy: 0.4488
Epoch 7/10
152/152 [==============================] - 86s 565ms/step - loss: 1.0066 - accuracy: 0.4820 - val_loss: 1.0049 - val_accuracy: 0.4

In [ ]:
#evaluating the model on test data
clf.evaluate([X_test_in, X_test_mask], y_test_in)

127/127 [==============================] - 27s 210ms/step - loss: 0.9568 - accuracy: 0.5718


[0.9567728042602539, 0.5717809796333313]

In [ ]:
clf.training = False
#  predictions on the testing data
y_pred = clf.predict([X_test_in, X_test_mask])

127/127 [==============================] - 25s 192ms/step


In [ ]:
y_pred_out = tf.math.argmax(y_pred, axis=-1) #Compute the argmax of the predictions along the last axis to get class labels
y_pred_out

<tf.Tensor: shape=(4054,), dtype=int64, numpy=array([1, 0, 0, ..., 2, 0, 2])>

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test_in, y_pred_out))

              precision    recall  f1-score   support

           0       0.52      0.32      0.39      1351
           1       0.63      0.48      0.54      1351
           2       0.57      0.92      0.70      1352

    accuracy                           0.57      4054
   macro avg       0.57      0.57      0.55      4054
weighted avg       0.57      0.57      0.55      4054



The accuracy of the model is 57% and the the model precsion for class 0(negative) is 52%, and class 2 i.e(positive) is 57% and finally precession to find neutral is 63% which means there is room for improvment and one can further fine tune the model.